# **Name: Ujjwal Kishor Sahoo | Roll Number: 21293 | NLP-Assignment 2**

In [1]:
!pip install keras-tuner --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.7 MB/s eta 0:00:00


## **Loading the dataset**

In [2]:
!wget https://raw.githubusercontent.com/islnlp/Assignment_1_2025/refs/heads/main/hate/train.csv
!wget https://raw.githubusercontent.com/islnlp/Assignment_1_2025/refs/heads/main/hate/val.csv
!wget https://raw.githubusercontent.com/islnlp/Assignment_1_2025/refs/heads/main/humor/train.csv
!wget https://raw.githubusercontent.com/islnlp/Assignment_1_2025/refs/heads/main/humor/val.csv
!wget https://raw.githubusercontent.com/islnlp/Assignment_1_2025/refs/heads/main/sarcasm/train.csv
!wget https://raw.githubusercontent.com/islnlp/Assignment_1_2025/refs/heads/main/sarcasm/val.csv

--2025-04-04 15:31:34--  https://raw.githubusercontent.com/islnlp/Assignment_1_2025/refs/heads/main/hate/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 406615 (397K) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>] 397.08K  --.-KB/s    in 0.03s   

2025-04-04 15:31:35 (13.5 MB/s) - ‘train.csv’ saved [406615/406615]

--2025-04-04 15:31:35--  https://raw.githubusercontent.com/islnlp/Assignment_1_2025/refs/heads/main/hate/val.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49821 (49K)

In [3]:
pip install keras

In [4]:
# Data manipulation and analysis
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report, f1_score

# Machine learning and neural networks
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Embedding, Dense, Flatten, Input, Concatenate, Dropout)
from tensorflow.keras.optimizers import Adam, AdamW, SGD
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import BinaryCrossentropy
import tensorflow.keras.backend as K
from keras.metrics import Precision, Recall
from kerastuner import HyperParameters
from kerastuner.tuners import RandomSearch
from keras_tuner import Objective
from sklearn.utils import class_weight



# Text processing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Hyperparameter tuning
import keras_tuner as kt
from keras_tuner import RandomSearch, Objective

<ipython-input-4-c829fb601b69>:18: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import HyperParameters


In [5]:
import spacy
nlp = spacy.load("en_core_web_sm")

# Hate Dataset

In [6]:
# Load the data
hate_train_data = pd.read_csv('/content/train.csv')
hate_test_data = pd.read_csv('/content/val.csv')

# Prepare the text and labels
texts = hate_train_data['Sentence'].values
labels = hate_train_data['Tag'].values

## **Pre-processing and Embedding**

In [13]:
def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    # Removing punctuation and special characters
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenization and lemmatization using spaCy
    doc = nlp(text)
    # Stopword removal and lemmatization
    text = re.sub(r"https\\S+|www\\S+", "", text)  # remove URLs
    text = re.sub(r"[^a-zA-Z0-9\\s]", "", text)  # remove special characters
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]

    return " ".join(tokens)

texts_preprocessed = [preprocess_text(text) for text in texts]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts_preprocessed)
vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(texts_preprocessed)

max_sequence_len = 128
X = pad_sequences(sequences, maxlen=max_sequence_len)

In [9]:
'''# Preparing dataset
sentences = [text.split() for text in texts_preprocessed]
vocab_size = len(tokenizer.word_index) + 1

# Define constants
embedding_dim = 100
window_size = 5  # For skip-gram context window

# Build a simple skip-gram pair generator function
def generate_skipgram_pairs(sentences, window_size, vocab_size):
    skipgrams = []
    for sentence in sentences:
        for i, word in enumerate(sentence):
            if word not in tokenizer.word_index:
                continue  # Skip unknown words
            target_word = tokenizer.word_index[word]
            context_window = sentence[max(i - window_size, 0): min(i + window_size + 1, len(sentence))]
            context_words = [
                tokenizer.word_index[w]
                for w in context_window
                if w != word and w in tokenizer.word_index
            ]
            for context_word in context_words:
                skipgrams.append([target_word, context_word])
    return np.array(skipgrams)

skipgrams = generate_skipgram_pairs(sentences, window_size, vocab_size)'''

'# Preparing dataset\nsentences = [text.split() for text in texts_preprocessed]\nvocab_size = len(tokenizer.word_index) + 1\n\n# Define constants\nembedding_dim = 100\nwindow_size = 5  # For skip-gram context window\n\n# Build a simple skip-gram pair generator function\ndef generate_skipgram_pairs(sentences, window_size, vocab_size):\n    skipgrams = []\n    for sentence in sentences:\n        for i, word in enumerate(sentence):\n            if word not in tokenizer.word_index:\n                continue  # Skip unknown words\n            target_word = tokenizer.word_index[word]\n            context_window = sentence[max(i - window_size, 0): min(i + window_size + 1, len(sentence))]\n            context_words = [\n                tokenizer.word_index[w]\n                for w in context_window\n                if w != word and w in tokenizer.word_index\n            ]\n            for context_word in context_words:\n                skipgrams.append([target_word, context_word])\n    return

In [10]:
'''# Splitting target words and context words
X_target, X_context = zip(*skipgrams)
X_target = np.array(X_target)
X_context = np.array(X_context)

# Defining the custom Word2Vec model using Keras
input_target = tf.keras.layers.Input(shape=(1,))
input_context = tf.keras.layers.Input(shape=(1,))

# Embedding layer for target and context
embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=1, name="embedding")
target_embedding = embedding(input_target)
context_embedding = embedding(input_context)'''

'# Splitting target words and context words\nX_target, X_context = zip(*skipgrams)\nX_target = np.array(X_target)\nX_context = np.array(X_context)\n\n# Defining the custom Word2Vec model using Keras\ninput_target = tf.keras.layers.Input(shape=(1,))\ninput_context = tf.keras.layers.Input(shape=(1,))\n\n# Embedding layer for target and context\nembedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=1, name="embedding")\ntarget_embedding = embedding(input_target)\ncontext_embedding = embedding(input_context)'

In [11]:
'''# Reshape embedding output for dot product calculation
target_embedding = tf.keras.layers.Reshape((embedding_dim,))(target_embedding)
context_embedding = tf.keras.layers.Reshape((embedding_dim,))(context_embedding)

# Compute dot product (cosine similarity between target and context)
dot_product = tf.keras.layers.Dot(axes=1)([target_embedding, context_embedding])
output = tf.keras.layers.Dense(1, activation='sigmoid')(dot_product)

# Define the model and compile it
word2vec_model = tf.keras.Model(inputs=[input_target, input_context], outputs=output)
word2vec_model.compile(optimizer='adam', loss='binary_crossentropy')'''

"# Reshape embedding output for dot product calculation\ntarget_embedding = tf.keras.layers.Reshape((embedding_dim,))(target_embedding)\ncontext_embedding = tf.keras.layers.Reshape((embedding_dim,))(context_embedding)\n\n# Compute dot product (cosine similarity between target and context)\ndot_product = tf.keras.layers.Dot(axes=1)([target_embedding, context_embedding])\noutput = tf.keras.layers.Dense(1, activation='sigmoid')(dot_product)\n\n# Define the model and compile it\nword2vec_model = tf.keras.Model(inputs=[input_target, input_context], outputs=output)\nword2vec_model.compile(optimizer='adam', loss='binary_crossentropy')"

In [12]:
'''# Prepare labels (1 for correct context, 0 for negative samples)
labels = np.ones((len(skipgrams), 1))  # Positive samples are labeled 1

# Train the model
word2vec_model.fit([X_target, X_context], labels, epochs=5, batch_size=128)

# Extract the trained word embeddings
trained_embeddings = word2vec_model.get_layer('embedding').get_weights()[0]

# Save the embeddings
np.save('word2vec_embeddings_hate.npy', trained_embeddings)'''

"# Prepare labels (1 for correct context, 0 for negative samples)\nlabels = np.ones((len(skipgrams), 1))  # Positive samples are labeled 1\n\n# Train the model\nword2vec_model.fit([X_target, X_context], labels, epochs=5, batch_size=128)\n\n# Extract the trained word embeddings\ntrained_embeddings = word2vec_model.get_layer('embedding').get_weights()[0]\n\n# Save the embeddings\nnp.save('word2vec_embeddings_hate.npy', trained_embeddings)"

In [ ]:
# Load the custom-trained Word2Vec embeddings
# trained_embeddings = np.load('word2vec_embeddings_hate.npy')

In [14]:
X = pad_sequences(sequences, maxlen=max_sequence_len)
labels = hate_train_data['Tag'].values

# Now perform train-test split
X_train, X_val, y_train, y_val = train_test_split(X, labels, test_size=0.1, random_state=60)


In [15]:
# Preprocess the test data
test_texts = hate_test_data['Sentence'].values
test_texts_preprocessed = [preprocess_text(text) for text in test_texts]
test_sequences = tokenizer.texts_to_sequences(test_texts_preprocessed)
X_test = pad_sequences(test_sequences, maxlen=max_sequence_len)

## **Defining custom evalutation metric**

In [16]:
# Define a custom macro F1 score function
def macro_f1_score(y_true, y_pred):
    # Ensure both y_true and y_pred are float32
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)

    # Convert predictions to binary (0 or 1)
    y_pred_bin = tf.round(y_pred)

    # Calculate precision, recall, and F1 score for each class
    def f1(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))

        precision = true_positives / (predicted_positives + K.epsilon())
        recall = true_positives / (possible_positives + K.epsilon())

        f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
        return f1_val

    f1_per_class = f1(y_true, y_pred_bin)
    return K.mean(f1_per_class)  # Macro F1 score (mean across all classes)

## **Training**

In [18]:
'''# Function to build the FFNN model for tuning
def build_model(hp):
    model = Sequential()

    embedding_dim = 100
    sequence_len = hp.Int('sequence_len', min_value=64, max_value=128, step=16)

    embedding_layer = Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        input_length=sequence_len,
        weights=[trained_embeddings],
        trainable=False
    )

    model.add(embedding_layer)
    model.add(Flatten())

    num_layers = hp.Int('num_layers', min_value=1, max_value=4)
    for i in range(num_layers):
        model.add(Dense(
            units=hp.Int(f'dense_units_{i+1}', min_value=32, max_value=64, step=16),
            activation='tanh'
        ))

    # Binary classification
    model.add(Dense(1, activation='sigmoid'))

    optimizer_choice = hp.Choice('optimizer', ['Adam', 'AdamW', 'SGD'])

    if optimizer_choice == 'Adam':
        opt = Adam(learning_rate=0.001)
    elif optimizer_choice == 'AdamW':
        opt = AdamW(learning_rate=0.001)
    else:
        opt = SGD(learning_rate=0.001)

    model.compile(
        optimizer=opt,
        loss=BinaryCrossentropy(),
        metrics=[macro_f1_score]
    )

    return model

# Define the objective using KerasTuner's Objective class
objective = Objective('val_macro_f1_score', direction='max')

# Hyperparameter search with Keras Tuner
tuner = RandomSearch(
    build_model,
    objective=objective,  # Explicitly specify the objective
    max_trials=15,  # Number of hyperparameter configurations to try
    executions_per_trial=1,  # Number of times to train each model configuration
    directory='hyperparam_tuning_hate',
    project_name='ffnn_tuning_v7.2213123123'
)

# Run the tuner search
tuner.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

# Get the best model
best_model_ffnn = tuner.get_best_models(num_models=1)[0]

# Summary of the best model
best_model_ffnn.summary()'''

"# Function to build the FFNN model for tuning\ndef build_model(hp):\n    model = Sequential()\n\n    embedding_dim = 100\n    sequence_len = hp.Int('sequence_len', min_value=64, max_value=128, step=16)\n\n    embedding_layer = Embedding(\n        input_dim=vocab_size,\n        output_dim=embedding_dim,\n        input_length=sequence_len,\n        weights=[trained_embeddings],\n        trainable=False\n    )\n\n    model.add(embedding_layer)\n    model.add(Flatten())\n\n    num_layers = hp.Int('num_layers', min_value=1, max_value=4)\n    for i in range(num_layers):\n        model.add(Dense(\n            units=hp.Int(f'dense_units_{i+1}', min_value=32, max_value=64, step=16),\n            activation='tanh'\n        ))\n\n    # Binary classification\n    model.add(Dense(1, activation='sigmoid'))\n\n    optimizer_choice = hp.Choice('optimizer', ['Adam', 'AdamW', 'SGD'])\n\n    if optimizer_choice == 'Adam':\n        opt = Adam(learning_rate=0.001)\n    elif optimizer_choice == 'AdamW':\n 

## **Saving the model**

In [23]:
# Save FFNN Model
# best_model_ffnn.save('best_model_ffnn_hate.keras')

In [29]:
!wget https://github.com/Ujjwal21293/DSE-318/raw/refs/heads/main/Assignment2/best_model_ffnn_hate.keras

--2025-04-04 15:48:16--  https://github.com/Ujjwal21293/DSE-318/raw/refs/heads/main/Assignment2/best_model_ffnn_hate.keras
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Ujjwal21293/DSE-318/refs/heads/main/Assignment2/best_model_ffnn_hate.keras [following]
--2025-04-04 15:48:17--  https://raw.githubusercontent.com/Ujjwal21293/DSE-318/refs/heads/main/Assignment2/best_model_ffnn_hate.keras
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8586315 (8.2M) [application/octet-stream]
Saving to: ‘best_model_ffnn_hate.keras’

best_model_ffnn_hat 100%[===================>]   8.19M  --.-KB/s    in 0.07s   

## **Loading the saved models**

In [30]:
# Load the model from the saved files
best_model_ffnn_hate = tf.keras.models.load_model('best_model_ffnn_hate.keras', custom_objects={'macro_f1_score': macro_f1_score})

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adamw', because it has 22 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [32]:
# Function to extract and print the macro avg F1-score
def print_macro_f1(classification_report_dict, model_name):
    macro_f1 = classification_report_dict['macro avg']['f1-score']
    print(f"{model_name} Model Macro Average F1-score: {macro_f1:.4f}")

## **Testing and Evaluation**

In [33]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Binary labels (0 or 1)
test_labels = hate_test_data['Tag'].values.reshape(-1, 1)

# Dictionary of models to evaluate
model = best_model_ffnn_hate

# Dictionary to store classification reports
reports = {}


predictions = model.predict(X_test)
predictions = (predictions > 0.5).astype(int)

    # Generate classification report
report = classification_report(
test_labels,
predictions,
target_names=['Non-Hate (0)', 'Hate (1)'],  # Optional: change labels as you wish
output_dict=True)

reports[model] = report

# Print macro F1 score and optionally others
f1 = f1_score(test_labels, predictions, average='macro')
acc = accuracy_score(test_labels, predictions)
print(f"{model} - Accuracy: {acc:.4f}, Macro F1: {f1:.4f}")

15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
<Sequential name=sequential, built=True> - Accuracy: 0.5689, Macro F1: 0.5592


# Humor Dataset

In [34]:
# Load the data
hate_train_data = pd.read_csv('/content/train.csv.1')
hate_test_data = pd.read_csv('/content/val.csv.1')

# Prepare the text and labels
texts = hate_train_data['Sentence'].values
labels = hate_train_data['Tag'].values

## **Pre-processing and Embedding**

In [35]:
def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    # Removing punctuation and special characters
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenization and lemmatization using spaCy
    doc = nlp(text)
    # Stopword removal and lemmatization
    text = re.sub(r"https\\S+|www\\S+", "", text)  # remove URLs
    text = re.sub(r"[^a-zA-Z0-9\\s]", "", text)  # remove special characters
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]

    return " ".join(tokens)

texts_preprocessed = [preprocess_text(text) for text in texts]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts_preprocessed)
vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(texts_preprocessed)

max_sequence_len = 128
X = pad_sequences(sequences, maxlen=max_sequence_len)

In [36]:
'''# Preparing dataset
sentences = [text.split() for text in texts_preprocessed]
vocab_size = len(tokenizer.word_index) + 1

# Define constants
embedding_dim = 100
window_size = 5  # For skip-gram context window

# Build a simple skip-gram pair generator function
def generate_skipgram_pairs(sentences, window_size, vocab_size):
    skipgrams = []
    for sentence in sentences:
        for i, word in enumerate(sentence):
            if word not in tokenizer.word_index:
                continue  # Skip unknown words
            target_word = tokenizer.word_index[word]
            context_window = sentence[max(i - window_size, 0): min(i + window_size + 1, len(sentence))]
            context_words = [
                tokenizer.word_index[w]
                for w in context_window
                if w != word and w in tokenizer.word_index
            ]
            for context_word in context_words:
                skipgrams.append([target_word, context_word])
    return np.array(skipgrams)

skipgrams = generate_skipgram_pairs(sentences, window_size, vocab_size)'''

'# Preparing dataset\nsentences = [text.split() for text in texts_preprocessed]\nvocab_size = len(tokenizer.word_index) + 1\n\n# Define constants\nembedding_dim = 100\nwindow_size = 5  # For skip-gram context window\n\n# Build a simple skip-gram pair generator function\ndef generate_skipgram_pairs(sentences, window_size, vocab_size):\n    skipgrams = []\n    for sentence in sentences:\n        for i, word in enumerate(sentence):\n            if word not in tokenizer.word_index:\n                continue  # Skip unknown words\n            target_word = tokenizer.word_index[word]\n            context_window = sentence[max(i - window_size, 0): min(i + window_size + 1, len(sentence))]\n            context_words = [\n                tokenizer.word_index[w]\n                for w in context_window\n                if w != word and w in tokenizer.word_index\n            ]\n            for context_word in context_words:\n                skipgrams.append([target_word, context_word])\n    return

In [37]:
'''# Splitting target words and context words
X_target, X_context = zip(*skipgrams)
X_target = np.array(X_target)
X_context = np.array(X_context)

# Defining the custom Word2Vec model using Keras
input_target = tf.keras.layers.Input(shape=(1,))
input_context = tf.keras.layers.Input(shape=(1,))

# Embedding layer for target and context
embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=1, name="embedding")
target_embedding = embedding(input_target)
context_embedding = embedding(input_context)'''

'# Splitting target words and context words\nX_target, X_context = zip(*skipgrams)\nX_target = np.array(X_target)\nX_context = np.array(X_context)\n\n# Defining the custom Word2Vec model using Keras\ninput_target = tf.keras.layers.Input(shape=(1,))\ninput_context = tf.keras.layers.Input(shape=(1,))\n\n# Embedding layer for target and context\nembedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=1, name="embedding")\ntarget_embedding = embedding(input_target)\ncontext_embedding = embedding(input_context)'

In [38]:
'''# Reshape embedding output for dot product calculation
target_embedding = tf.keras.layers.Reshape((embedding_dim,))(target_embedding)
context_embedding = tf.keras.layers.Reshape((embedding_dim,))(context_embedding)

# Compute dot product (cosine similarity between target and context)
dot_product = tf.keras.layers.Dot(axes=1)([target_embedding, context_embedding])
output = tf.keras.layers.Dense(1, activation='sigmoid')(dot_product)

# Define the model and compile it
word2vec_model = tf.keras.Model(inputs=[input_target, input_context], outputs=output)
word2vec_model.compile(optimizer='adam', loss='binary_crossentropy')'''

"# Reshape embedding output for dot product calculation\ntarget_embedding = tf.keras.layers.Reshape((embedding_dim,))(target_embedding)\ncontext_embedding = tf.keras.layers.Reshape((embedding_dim,))(context_embedding)\n\n# Compute dot product (cosine similarity between target and context)\ndot_product = tf.keras.layers.Dot(axes=1)([target_embedding, context_embedding])\noutput = tf.keras.layers.Dense(1, activation='sigmoid')(dot_product)\n\n# Define the model and compile it\nword2vec_model = tf.keras.Model(inputs=[input_target, input_context], outputs=output)\nword2vec_model.compile(optimizer='adam', loss='binary_crossentropy')"

In [39]:
'''# Prepare labels (1 for correct context, 0 for negative samples)
labels = np.ones((len(skipgrams), 1))  # Positive samples are labeled 1

# Train the model
word2vec_model.fit([X_target, X_context], labels, epochs=5, batch_size=128)

# Extract the trained word embeddings
trained_embeddings = word2vec_model.get_layer('embedding').get_weights()[0]

# Save the embeddings
np.save('word2vec_embeddings_humor.npy', trained_embeddings)'''

"# Prepare labels (1 for correct context, 0 for negative samples)\nlabels = np.ones((len(skipgrams), 1))  # Positive samples are labeled 1\n\n# Train the model\nword2vec_model.fit([X_target, X_context], labels, epochs=5, batch_size=128)\n\n# Extract the trained word embeddings\ntrained_embeddings = word2vec_model.get_layer('embedding').get_weights()[0]\n\n# Save the embeddings\nnp.save('word2vec_embeddings_humor.npy', trained_embeddings)"

In [40]:
# Load the custom-trained Word2Vec embeddings
# trained_embeddings = np.load('word2vec_embeddings_humor.npy')

In [41]:
X = pad_sequences(sequences, maxlen=max_sequence_len)
labels = hate_train_data['Tag'].values

# Now perform train-test split
X_train, X_val, y_train, y_val = train_test_split(X, labels, test_size=0.1, random_state=60)


In [42]:
# Preprocess the test data
test_texts = hate_test_data['Sentence'].values
test_texts_preprocessed = [preprocess_text(text) for text in test_texts]
test_sequences = tokenizer.texts_to_sequences(test_texts_preprocessed)
X_test = pad_sequences(test_sequences, maxlen=max_sequence_len)

## **Training**

In [43]:
'''# Function to build the FFNN model for tuning
def build_model(hp):
    model = Sequential()

    embedding_dim = 100
    sequence_len = hp.Int('sequence_len', min_value=64, max_value=128, step=16)

    embedding_layer = Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        input_length=sequence_len,
        weights=[trained_embeddings],
        trainable=False
    )

    model.add(embedding_layer)
    model.add(Flatten())

    num_layers = hp.Int('num_layers', min_value=1, max_value=4)
    for i in range(num_layers):
        model.add(Dense(
            units=hp.Int(f'dense_units_{i+1}', min_value=32, max_value=64, step=16),
            activation='tanh'
        ))

    # Binary classification
    model.add(Dense(1, activation='sigmoid'))

    optimizer_choice = hp.Choice('optimizer', ['Adam', 'AdamW', 'SGD'])

    if optimizer_choice == 'Adam':
        opt = Adam(learning_rate=0.001)
    elif optimizer_choice == 'AdamW':
        opt = AdamW(learning_rate=0.001)
    else:
        opt = SGD(learning_rate=0.001)

    model.compile(
        optimizer=opt,
        loss=BinaryCrossentropy(),
        metrics=[macro_f1_score]
    )

    return model

# Define the objective using KerasTuner's Objective class
objective = Objective('val_macro_f1_score', direction='max')

# Hyperparameter search with Keras Tuner
tuner = RandomSearch(
    build_model,
    objective=objective,  # Explicitly specify the objective
    max_trials=15,  # Number of hyperparameter configurations to try
    executions_per_trial=1,  # Number of times to train each model configuration
    directory='hyperparam_tuning_humor',
    project_name='ffnn_tuning_v7.2213123123'
)

# Run the tuner search
tuner.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

# Get the best model
best_model_ffnn = tuner.get_best_models(num_models=1)[0]

# Summary of the best model
best_model_ffnn.summary()'''

"# Function to build the FFNN model for tuning\ndef build_model(hp):\n    model = Sequential()\n\n    embedding_dim = 100\n    sequence_len = hp.Int('sequence_len', min_value=64, max_value=128, step=16)\n\n    embedding_layer = Embedding(\n        input_dim=vocab_size,\n        output_dim=embedding_dim,\n        input_length=sequence_len,\n        weights=[trained_embeddings],\n        trainable=False\n    )\n\n    model.add(embedding_layer)\n    model.add(Flatten())\n\n    num_layers = hp.Int('num_layers', min_value=1, max_value=4)\n    for i in range(num_layers):\n        model.add(Dense(\n            units=hp.Int(f'dense_units_{i+1}', min_value=32, max_value=64, step=16),\n            activation='tanh'\n        ))\n\n    # Binary classification\n    model.add(Dense(1, activation='sigmoid'))\n\n    optimizer_choice = hp.Choice('optimizer', ['Adam', 'AdamW', 'SGD'])\n\n    if optimizer_choice == 'Adam':\n        opt = Adam(learning_rate=0.001)\n    elif optimizer_choice == 'AdamW':\n 

## **Saving the model**

In [44]:
# Save FFNN Model
# best_model_ffnn.save('best_model_ffnn_humor.keras')

In [45]:
!wget https://github.com/Ujjwal21293/DSE-318/raw/refs/heads/main/Assignment2/best_model_ffnn_humor.keras

--2025-04-04 15:50:56--  https://github.com/Ujjwal21293/DSE-318/raw/refs/heads/main/Assignment2/best_model_ffnn_humor.keras
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Ujjwal21293/DSE-318/refs/heads/main/Assignment2/best_model_ffnn_humor.keras [following]
--2025-04-04 15:50:57--  https://raw.githubusercontent.com/Ujjwal21293/DSE-318/refs/heads/main/Assignment2/best_model_ffnn_humor.keras
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5316506 (5.1M) [application/octet-stream]
Saving to: ‘best_model_ffnn_humor.keras’

best_model_ffnn_hum 100%[===================>]   5.07M  --.-KB/s    in 0.06

## **Loading the saved models**

In [46]:
# Load the model from the saved files
best_model_ffnn = tf.keras.models.load_model('best_model_ffnn_humor.keras', custom_objects={'macro_f1_score': macro_f1_score})

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adamw', because it has 22 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [47]:
# Function to extract and print the macro avg F1-score
def print_macro_f1(classification_report_dict, model_name):
    macro_f1 = classification_report_dict['macro avg']['f1-score']
    print(f"{model_name} Model Macro Average F1-score: {macro_f1:.4f}")

## **Testing and Evaluation**

In [48]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Binary labels (0 or 1)
test_labels = hate_test_data['Tag'].values.reshape(-1, 1)

# Dictionary of models to evaluate
model = best_model_ffnn

# Dictionary to store classification reports
reports = {}


predictions = model.predict(X_test)
predictions = (predictions > 0.5).astype(int)

    # Generate classification report
report = classification_report(
test_labels,
predictions,
target_names=['Non-Hate (0)', 'Hate (1)'],  # Optional: change labels as you wish
output_dict=True)

reports[model] = report

# Print macro F1 score and optionally others
f1 = f1_score(test_labels, predictions, average='macro')
acc = accuracy_score(test_labels, predictions)
print(f"{model} - Accuracy: {acc:.4f}, Macro F1: {f1:.4f}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
<Sequential name=sequential, built=True> - Accuracy: 0.5966, Macro F1: 0.3737


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Sarcasm Dataset

In [49]:
# Load the data
hate_train_data = pd.read_csv('/content/train.csv.2')
hate_test_data = pd.read_csv('/content/val.csv.2')

# Prepare the text and labels
texts = hate_train_data['Sentence'].values
labels = hate_train_data['Tag'].values

## **Pre-processing and Embedding**

In [50]:
def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    # Removing punctuation and special characters
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenization and lemmatization using spaCy
    doc = nlp(text)
    # Stopword removal and lemmatization
    text = re.sub(r"https\\S+|www\\S+", "", text)  # remove URLs
    text = re.sub(r"[^a-zA-Z0-9\\s]", "", text)  # remove special characters
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]

    return " ".join(tokens)

texts_preprocessed = [preprocess_text(text) for text in texts]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts_preprocessed)
vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(texts_preprocessed)

max_sequence_len = 128
X = pad_sequences(sequences, maxlen=max_sequence_len)

In [51]:
'''# Preparing dataset
sentences = [text.split() for text in texts_preprocessed]
vocab_size = len(tokenizer.word_index) + 1

# Define constants
embedding_dim = 100
window_size = 5  # For skip-gram context window

# Build a simple skip-gram pair generator function
def generate_skipgram_pairs(sentences, window_size, vocab_size):
    skipgrams = []
    for sentence in sentences:
        for i, word in enumerate(sentence):
            if word not in tokenizer.word_index:
                continue  # Skip unknown words
            target_word = tokenizer.word_index[word]
            context_window = sentence[max(i - window_size, 0): min(i + window_size + 1, len(sentence))]
            context_words = [
                tokenizer.word_index[w]
                for w in context_window
                if w != word and w in tokenizer.word_index
            ]
            for context_word in context_words:
                skipgrams.append([target_word, context_word])
    return np.array(skipgrams)

skipgrams = generate_skipgram_pairs(sentences, window_size, vocab_size)'''

'# Preparing dataset\nsentences = [text.split() for text in texts_preprocessed]\nvocab_size = len(tokenizer.word_index) + 1\n\n# Define constants\nembedding_dim = 100\nwindow_size = 5  # For skip-gram context window\n\n# Build a simple skip-gram pair generator function\ndef generate_skipgram_pairs(sentences, window_size, vocab_size):\n    skipgrams = []\n    for sentence in sentences:\n        for i, word in enumerate(sentence):\n            if word not in tokenizer.word_index:\n                continue  # Skip unknown words\n            target_word = tokenizer.word_index[word]\n            context_window = sentence[max(i - window_size, 0): min(i + window_size + 1, len(sentence))]\n            context_words = [\n                tokenizer.word_index[w]\n                for w in context_window\n                if w != word and w in tokenizer.word_index\n            ]\n            for context_word in context_words:\n                skipgrams.append([target_word, context_word])\n    return

In [52]:
'''# Splitting target words and context words
X_target, X_context = zip(*skipgrams)
X_target = np.array(X_target)
X_context = np.array(X_context)

# Defining the custom Word2Vec model using Keras
input_target = tf.keras.layers.Input(shape=(1,))
input_context = tf.keras.layers.Input(shape=(1,))

# Embedding layer for target and context
embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=1, name="embedding")
target_embedding = embedding(input_target)
context_embedding = embedding(input_context)'''

'# Splitting target words and context words\nX_target, X_context = zip(*skipgrams)\nX_target = np.array(X_target)\nX_context = np.array(X_context)\n\n# Defining the custom Word2Vec model using Keras\ninput_target = tf.keras.layers.Input(shape=(1,))\ninput_context = tf.keras.layers.Input(shape=(1,))\n\n# Embedding layer for target and context\nembedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=1, name="embedding")\ntarget_embedding = embedding(input_target)\ncontext_embedding = embedding(input_context)'

In [53]:
'''# Reshape embedding output for dot product calculation
target_embedding = tf.keras.layers.Reshape((embedding_dim,))(target_embedding)
context_embedding = tf.keras.layers.Reshape((embedding_dim,))(context_embedding)

# Compute dot product (cosine similarity between target and context)
dot_product = tf.keras.layers.Dot(axes=1)([target_embedding, context_embedding])
output = tf.keras.layers.Dense(1, activation='sigmoid')(dot_product)

# Define the model and compile it
word2vec_model = tf.keras.Model(inputs=[input_target, input_context], outputs=output)
word2vec_model.compile(optimizer='adam', loss='binary_crossentropy')'''

"# Reshape embedding output for dot product calculation\ntarget_embedding = tf.keras.layers.Reshape((embedding_dim,))(target_embedding)\ncontext_embedding = tf.keras.layers.Reshape((embedding_dim,))(context_embedding)\n\n# Compute dot product (cosine similarity between target and context)\ndot_product = tf.keras.layers.Dot(axes=1)([target_embedding, context_embedding])\noutput = tf.keras.layers.Dense(1, activation='sigmoid')(dot_product)\n\n# Define the model and compile it\nword2vec_model = tf.keras.Model(inputs=[input_target, input_context], outputs=output)\nword2vec_model.compile(optimizer='adam', loss='binary_crossentropy')"

In [54]:
'''# Prepare labels (1 for correct context, 0 for negative samples)
labels = np.ones((len(skipgrams), 1))  # Positive samples are labeled 1

# Train the model
word2vec_model.fit([X_target, X_context], labels, epochs=5, batch_size=128)

# Extract the trained word embeddings
trained_embeddings = word2vec_model.get_layer('embedding').get_weights()[0]

# Save the embeddings
np.save('word2vec_embeddings_sarcasm.npy', trained_embeddings)'''

"# Prepare labels (1 for correct context, 0 for negative samples)\nlabels = np.ones((len(skipgrams), 1))  # Positive samples are labeled 1\n\n# Train the model\nword2vec_model.fit([X_target, X_context], labels, epochs=5, batch_size=128)\n\n# Extract the trained word embeddings\ntrained_embeddings = word2vec_model.get_layer('embedding').get_weights()[0]\n\n# Save the embeddings\nnp.save('word2vec_embeddings_sarcasm.npy', trained_embeddings)"

In [55]:
# Load the custom-trained Word2Vec embeddings
# trained_embeddings = np.load('word2vec_embeddings_sarcasm.npy')

In [56]:
X = pad_sequences(sequences, maxlen=max_sequence_len)
labels = hate_train_data['Tag'].values

# Now perform train-test split
X_train, X_val, y_train, y_val = train_test_split(X, labels, test_size=0.1, random_state=60)

In [57]:
# Preprocess the test data
test_texts = hate_test_data['Sentence'].values
test_texts_preprocessed = [preprocess_text(text) for text in test_texts]
test_sequences = tokenizer.texts_to_sequences(test_texts_preprocessed)
X_test = pad_sequences(test_sequences, maxlen=max_sequence_len)

## **Training**

In [59]:
'''# Function to build the FFNN model for tuning
def build_model(hp):
    model = Sequential()

    embedding_dim = 100
    sequence_len = hp.Int('sequence_len', min_value=64, max_value=128, step=16)

    embedding_layer = Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        input_length=sequence_len,
        weights=[trained_embeddings],
        trainable=False
    )

    model.add(embedding_layer)
    model.add(Flatten())

    num_layers = hp.Int('num_layers', min_value=1, max_value=4)
    for i in range(num_layers):
        model.add(Dense(
            units=hp.Int(f'dense_units_{i+1}', min_value=32, max_value=64, step=16),
            activation='tanh'
        ))

    # Binary classification
    model.add(Dense(1, activation='sigmoid'))

    optimizer_choice = hp.Choice('optimizer', ['Adam', 'AdamW', 'SGD'])

    if optimizer_choice == 'Adam':
        opt = Adam(learning_rate=0.001)
    elif optimizer_choice == 'AdamW':
        opt = AdamW(learning_rate=0.001)
    else:
        opt = SGD(learning_rate=0.001)

    model.compile(
        optimizer=opt,
        loss=BinaryCrossentropy(),
        metrics=[macro_f1_score]
    )

    return model

# Define the objective using KerasTuner's Objective class
objective = Objective('val_macro_f1_score', direction='max')

# Hyperparameter search with Keras Tuner
tuner = RandomSearch(
    build_model,
    objective=objective,  # Explicitly specify the objective
    max_trials=15,  # Number of hyperparameter configurations to try
    executions_per_trial=1,  # Number of times to train each model configuration
    directory='hyperparam_tuning_sarcasm',
    project_name='ffnn_tuning_v7.2213123123'
)

# Run the tuner search
tuner.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

# Get the best model
best_model_ffnn = tuner.get_best_models(num_models=1)[0]

# Summary of the best model
best_model_ffnn.summary()'''

"# Function to build the FFNN model for tuning\ndef build_model(hp):\n    model = Sequential()\n\n    embedding_dim = 100\n    sequence_len = hp.Int('sequence_len', min_value=64, max_value=128, step=16)\n\n    embedding_layer = Embedding(\n        input_dim=vocab_size,\n        output_dim=embedding_dim,\n        input_length=sequence_len,\n        weights=[trained_embeddings],\n        trainable=False\n    )\n\n    model.add(embedding_layer)\n    model.add(Flatten())\n\n    num_layers = hp.Int('num_layers', min_value=1, max_value=4)\n    for i in range(num_layers):\n        model.add(Dense(\n            units=hp.Int(f'dense_units_{i+1}', min_value=32, max_value=64, step=16),\n            activation='tanh'\n        ))\n\n    # Binary classification\n    model.add(Dense(1, activation='sigmoid'))\n\n    optimizer_choice = hp.Choice('optimizer', ['Adam', 'AdamW', 'SGD'])\n\n    if optimizer_choice == 'Adam':\n        opt = Adam(learning_rate=0.001)\n    elif optimizer_choice == 'AdamW':\n 

## **Saving the model**

In [60]:
# Save FFNN Model
# best_model_ffnn.save('best_model_ffnn_sarcasm.keras')

In [61]:
!wget https://github.com/Ujjwal21293/DSE-318/raw/refs/heads/main/Assignment2/best_model_ffnn_sarcasm.keras

--2025-04-04 15:52:38--  https://github.com/Ujjwal21293/DSE-318/raw/refs/heads/main/Assignment2/best_model_ffnn_sarcasm.keras
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Ujjwal21293/DSE-318/refs/heads/main/Assignment2/best_model_ffnn_sarcasm.keras [following]
--2025-04-04 15:52:38--  https://raw.githubusercontent.com/Ujjwal21293/DSE-318/refs/heads/main/Assignment2/best_model_ffnn_sarcasm.keras
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8319259 (7.9M) [application/octet-stream]
Saving to: ‘best_model_ffnn_sarcasm.keras’

best_model_ffnn_sar 100%[===================>]   7.93M  --.-KB/s   

## **Loading the saved models**

In [62]:
# Load the model from the saved files
best_model_ffnn = tf.keras.models.load_model('best_model_ffnn_sarcasm.keras', custom_objects={'macro_f1_score': macro_f1_score})

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adamw', because it has 18 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [63]:
# Function to extract and print the macro avg F1-score
def print_macro_f1(classification_report_dict, model_name):
    macro_f1 = classification_report_dict['macro avg']['f1-score']
    print(f"{model_name} Model Macro Average F1-score: {macro_f1:.4f}")

## **Testing and Evaluation**

In [64]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Binary labels (0 or 1)
test_labels = hate_test_data['Tag'].values.reshape(-1, 1)

# Dictionary of models to evaluate
model = best_model_ffnn

# Dictionary to store classification reports
reports = {}


predictions = model.predict(X_test)
predictions = (predictions > 0.5).astype(int)

    # Generate classification report
report = classification_report(
test_labels,
predictions,
target_names=['Non-Hate (0)', 'Hate (1)'],  # Optional: change labels as you wish
output_dict=True)

reports[model] = report

# Print macro F1 score and optionally others
f1 = f1_score(test_labels, predictions, average='macro')
acc = accuracy_score(test_labels, predictions)
print(f"{model} - Accuracy: {acc:.4f}, Macro F1: {f1:.4f}")

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
<Sequential name=sequential, built=True> - Accuracy: 0.8419, Macro F1: 0.7073
